# 🚀 Regelbasierte OCR-Nachbearbeitung

Listen wir einige typische Fehler in unserem Korpus auf:

* "fie" statt "sie" (Bild und Ergebnis später hinzufügen)
* "vm", "vnd" statt "um", "und"
* "<" statt "ch"

Einige Dinge sind keine Fehler, sondern Merkmale der historischen Orthographie, die wir für die weitere Verarbeitung mit modernen NLP-Tools normalisieren möchten:

* "ſ" statt "s"

<!--
Let's list some typical regular errors in our corpus:

* "fie" statt "sie" (add pic & result later)
* "vm", "vnd" statt "um", "und"
* "<" statt "ch"

Some things are not errors, but rather features of historical orthograpy which we would like to normalise for further processing with modern NLP tools:

* "ſ" statt "s" -->

In vielen Fällen können wir dies mit einigen regulären Such- und Ersetzungsmustern beheben (z.B. jedes `<`, das nicht von Leerzeichen umgeben ist, in `ch` umwandeln).

Der Standardweg, solche Muster auf einem Computer auszudrücken und zu implementieren, sind reguläre Ausdrücke. Mehr über reguläre Ausdrücke erfahren Sie [hier](https://www.w3schools.com/python/python_regex.asp).

<!-- In many cases we can fix it with some regular search-and replace patterns (e.g. take each `<` not surrounded by spaces and convert into `ch`)

The standard way to express & implement such patterns on a computer would be regular expressions. You can learn more about regular expressions [here](https://www.w3schools.com/python/python_regex.asp). -->

## Implementierung von Regeln für typische Fehler mit regulären Ausdrücken

In [ ]:
import re

In [ ]:
def post_correct_text(ocr_output):
    cleaner_output = re.sub(r'(\w)<(\w)', '\\1ch\\2', ocr_output)
    cleaner_output = re.sub(r'(\w)5(\w)', '\\1s\\2', cleaner_output)
    cleaner_output = re.sub(r'\bv(m|nd)\b', 'u\\1', cleaner_output)
    cleaner_output = re.sub(r'\bfie\b', 'sie', cleaner_output)
    cleaner_output = cleaner_output.replace('ſ','s')
    cleaner_output = cleaner_output.replace('\n',' ')
    return cleaner_output

## Anwendung der Regeln auf die OCR-Ergebnisse <!-- ## Applying rules to OCR results --> 

In [ ]:
import pytesseract
from PIL import Image
from auxiliary.measure_ocr_quality import measure_ocr_quality

<img src="grippe.jpeg" width=700>

In [ ]:
ocr_output = pytesseract.image_to_string(Image.open('grippe.jpeg'), lang='frk')

In [ ]:
print(ocr_output)

In [ ]:
ocr_output_corr = post_correct_text(ocr_output)

Lassen Sie uns sehen, wie sich das Ganze verändert hat: <!-- Let us see how the whole thing changed: --> 

In [ ]:
print(ocr_output_corr)

(messung-der-verbesserung)=
## Messung der Verbesserung

Lassen Sie uns sehen, wie sich die regelbasierte Nachkorrektur auf die OCR-Qualitätsmetriken ausgewirkt hat
<!-- Let us see how the post-correction affected the OCR quality metrics --> 

In [ ]:
ground_truth = 'Die Grippe wütet weiter. Zunahme der schweren Fälle in Berlin. Die Zahl der Grippefälle ist in den letzten Tagen auch in Groß-Berlin noch erheblich gestiegen. Die Warenhäuser und sonstigen großen Geschäfte, die Kriegs- und die privaten Betriebe klagen, daß übermäßig viele Angestellte sich haben krank melden müssen und auch bei der Post und bei der Straßenbahn ist der Prozentsatz der Grippekranken deutlich gestiegen.'

### Originales (unkorrigiertes) OCR-Ergebnis

In [ ]:
precision, recall, f_score = measure_ocr_quality(ocr_output, ground_truth)

In [ ]:
print(f'Precision: {round(precision, 4)}\nRecall: {round(recall, 4)}\nF1-score: {round(f_score, 4)}')

### Korrigiertes OCR-Ergebnis

In [ ]:
precision, recall, f_score = measure_ocr_quality(ocr_output_corr, ground_truth)

In [ ]:
print(f'Precision: {round(precision, 4)}\nRecall: {round(recall, 4)}\nF1-score: {round(f_score, 4)}')

Also, unsere F-Measure hat sich etwas verbessert, gut! <!-- So, our F-measure increased a bit, good! -->

## (Advanced) Ausführung des regelbasierten OCR-Nachkorrekturverfahrens auf dem gesamten Korpus

In [ ]:
from pathlib import Path
from tqdm import tqdm

In [ ]:
pathtxt = Path('../data/txt')

for file in tqdm(pathtxt.iterdir()):
    if file.suffix == '.txt':
        text = file.read_text()
        corrected = post_correct_text(text)
        file.write_text(corrected)